In [1]:
import pickle
import warnings
import pandas as pd
from tqdm import tqdm
from datetime import datetime

from utils.ml_train import train_models
from utils.performance import calculate_metrics

warnings.filterwarnings('ignore')

month = datetime.now().date().month
day = datetime.now().date().day

### Select feature set & data regime

In [ ]:
#Either "DRFP" or "QM"
features = "DRFP"
#Exclusion of JnJ25 in Train data can be set to True or False
no_jjhte_train = False

In [4]:
with open("data/HTE_BH_10_iter_9-3_Clusters_v2025-11.pkl", 'rb') as f:
    df_HTE_BH = pickle.load(f)
       
model_names = ["rf_model", "gradient_boosting_model", "lr_model", "mlp_model", "knn_model", "gaussianNB_model"] 
metrics_names = ['ROC AUC', 'Balanced Accuracy', 'F1 Score', 'F0 Score',
                 'AU-PR-C', 'Precision 1', 'Recall 1']
sel_metrics = ["ROC AUC", "F1 Score"]

all_with_hte_results_list = []
conf_bucket_all_results = []
for rand_state in tqdm(range(10)):    
    with_hte_results, test_sets, per_bucket_avg_results = train_test_ood_cluster_static(df_HTE_BH, model_names, metrics_names, rand_state, features = features, 
                                                                                        no_jjhte_train = no_jjhte_train, reverse_clusters = False, sample_train = 1)
    with_hte_results_copy = copy.deepcopy(with_hte_results)
    all_with_hte_results_list.append(with_hte_results_copy)
    conf_bucket_all_results.append(per_bucket_avg_results)

metrics_names = [m + "_avg" for m in metrics_names]

all_with_hte_results, best_model = calculate_model_performance(all_with_hte_results_list, metrics_names, sel_metrics)
final_per_bucket_results = combine_per_bucket_results(conf_bucket_all_results)

for col in all_with_hte_results.columns:
    if col.count('_') == 1:
        all_with_hte_results.rename(columns={col : col.split("_")[0]+" "+col.split(" ")[-1]}, inplace = True)
    elif col.count('_') == 2:
        all_with_hte_results.rename(columns={col : col.split("_")[0]+"_"+col.split("_")[1]+" "+col.split(" ")[-1]}, inplace = True)   
        
with open(f"results/JnJ25_{not(no_jjhte_train)}_Train_{features}_all_{year}-{month}-{day}.pkl", 'wb') as f:
    pickle.dump(all_with_hte_results_list, f)
    
all_with_hte_results.to_csv(f"results/JnJ25_{not(no_jjhte_train)}_Train_{features}_best_{year}-{month}-{day}.csv")

100%|██████████| 10/10 [17:52<00:00, 107.26s/it]
